# Model Training
,

cells
metadata
source
Train XGBoost and LSTM models, plot training curves.

In [3]:
import os
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Input

FEATURES = [
    "latitude",
    "longitude",
    "depth_km",
    "prev_magnitude",
    "quake_count_7d",
    "quake_count_30d",
    "avg_magnitude_30d",
    "max_magnitude_30d",
    "days_since_last_quake",
    "month",
    "seismic_zone",
]

base_dir = Path("..").resolve()
if "COLAB_RELEASE_TAG" in os.environ:
    repo_url = "https://github.com/vsiva763-git/Earthquake-alert-system.git"
    repo_dir = Path("/content/Earthquake-alert-system")
    if not repo_dir.exists():
        import subprocess
        subprocess.run(["git", "clone", repo_url, str(repo_dir)], check=True)
    base_dir = repo_dir / "earthquake_model"
data_path = base_dir / "data" / "processed" / "features.csv"

df = pd.read_csv(data_path, parse_dates=["time"])
print(f"✓ Data loaded: {len(df)} samples, {len(df.columns)} features")
df.head()

✓ Data loaded: 12356 samples, 14 features


,time,latitude,longitude,depth_km,magnitude,place,seismic_zone,month,prev_magnitude,days_since_last_quake,quake_count_7d,quake_count_30d,avg_magnitude_30d,max_magnitude_30d
0,2006-02-22 22:59:37.180000+00:00,34.626,73.499,10.0,4.3,"28 km N of Muzaffar?b?d, Pakistan",5,2,0.0,0.000000,0,0,0.00,0.0
1,2006-02-23 05:03:06.110000+00:00,36.227,71.399,10.0,3.9,"52 km SSW of Ashk?sham, Afghanistan",2,2,0.0,0.000000,0,0,0.00,0.0
2,2006-02-23 20:04:53.550000+00:00,26.912,91.705,10.0,5.8,"23 km ENE of Samdrup Jongkhar, Bhutan",5,2,4.3,0.878662,0,1,4.30,4.3
3,2006-02-23 20:07:26.170000+00:00,26.863,91.632,10.0,5.2,"14 km ENE of Samdrup Jongkhar, Bhutan",5,2,5.8,0.001766,1,2,5.05,5.8
4,2006-02-23 20:22:46.550000+00:00,26.830,91.809,10.0,3.9,"30 km WNW of Udalguri, India",5,2,5.2,0.010653,2,3,5.10,5.8


In [4]:
import sys
sys.path.insert(0, str(base_dir))
from src.alert_classifier import classify_alert

# Convert magnitudes to alert labels (LOW/MID/HIGH) with class weight balancing
y_labels = [classify_alert(m) for m in df["magnitude"].values]
label_to_int = {"LOW": 0, "MID": 1, "HIGH": 2}
y_alert = np.array([label_to_int[label] for label in y_labels])

X = df[FEATURES].fillna(0.0)
X_train, X_test, y_train_alert, y_test_alert = train_test_split(
    X, y_alert, test_size=0.2, random_state=42
)

# Calculate scale_pos_weight for the minority HIGH class (class 2)
scale_pos_weight = len(y_train_alert[y_train_alert != 2]) / len(y_train_alert[y_train_alert == 2])

from xgboost import XGBClassifier

xgb = XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    objective="multi:softmax",
    num_class=3,
    random_state=42,
)
xgb.fit(X_train, y_train_alert)

models_dir = base_dir / "models"
models_dir.mkdir(exist_ok=True)
xgb.save_model(models_dir / "xgb_model.json")
print(f"✓ XGBoost Classification Model trained with scale_pos_weight={scale_pos_weight:.2f}")

/workspaces/Earthquake-alert-system/.venv/lib/python3.12/site-packages/xgboost/training.py:200: UserWarning: [15:34:53] WARNING: /__w/xgboost/xgboost/src/learner.cc:782: 
Parameters: { "scale_pos_weight" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


✓ XGBoost Classification Model trained with scale_pos_weight=51.57


In [ ]:
import numpy as np

sequence_len = 10
seq_features = ["latitude", "longitude", "depth_km", "magnitude", "days_since_last_quake", "seismic_zone"]

sequences = []
targets = []
for _, zone_df in df.groupby("seismic_zone"):
    zone_df = zone_df.sort_values("time")
    data = zone_df[seq_features].values
    for i in range(sequence_len, len(data)):
        sequences.append(data[i - sequence_len : i])
        targets.append(zone_df.iloc[i]["magnitude"])

sequences = np.array(sequences)
targets = np.array(targets)

X_train, X_test, y_train, y_test = train_test_split(
    sequences, targets, test_size=0.2, random_state=42
)

from tensorflow.keras.layers import Input

lstm = Sequential([
    Input(shape=(sequence_len, len(seq_features))),
    LSTM(32),
    Dense(16, activation="relu"),
    Dense(1)
])
lstm.compile(optimizer="adam", loss="mse", metrics=["mae"])
history = lstm.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=32)

models_dir = base_dir / "models"
models_dir.mkdir(exist_ok=True)
lstm.save(models_dir / "lstm_model.keras")

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(history.history["loss"], label="train")
plt.plot(history.history["val_loss"], label="val")
plt.title("LSTM Training Loss")
plt.xlabel("Epoch")
plt.ylabel("MSE")
plt.legend()
plt.tight_layout()